In [1]:
import tensorflow as tf

In [2]:
import numpy
import os
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from alibi.explainers import IntegratedGradients

In [3]:
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
%set_env CUDA_VISIBLE_DEVICES=2
tf.test.is_gpu_available()

env: CUDA_VISIBLE_DEVICES=2


True

In [5]:
NUM_WORDS=1000 # only use top 1000 words
INDEX_FROM=3   # word index offset

word_to_id = tf.keras.datasets.imdb.get_word_index()
word_to_id = {k:(v+INDEX_FROM) for k,v in word_to_id.items()}
word_to_id["<PAD>"] = 0
word_to_id["<START>"] = 1
word_to_id["<UNK>"] = 2
word_to_id["<UNUSED>"] = 3

id_to_word = {value:key for key,value in word_to_id.items()}

In [6]:
max_features = 20000
# cut texts after this number of words (among top max_features most common words)
maxlen = 100
batch_size = 32

print('Loading data...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)


Loading data...
25000 train sequences
25000 test sequences
Pad sequences (samples x time)
x_train shape: (25000, 100)
x_test shape: (25000, 100)


In [7]:
print(' '.join(id_to_word[id] for id in x_train[10] ))

corner and this helps to ensure that maléfique actually does manage to be quite frightening the film is memorable for a lot of reasons outside the central plot the characters are all very interesting in their own way and the fact that the book itself almost takes on its own character is very well done anyone worried that the film won't deliver by the end won't be disappointed either as the ending both makes sense and manages to be quite horrifying overall maléfique is a truly great horror film and one of the best of the decade highly recommended viewing


# Model

In [8]:
load_model = True

In [9]:
filepath = './model_imdb/'  # change to directory where model is downloaded
if load_model:
    model = tf.keras.models.load_model(os.path.join(filepath, 'model.h5'))
else:
    print('Build model...')
    model = Sequential()
    model.add(Embedding(max_features, 128))
    model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(1, activation='sigmoid'))

    # try using different optimizers and different optimizer configs
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    print('Train...')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=1,
              validation_data=(x_test, y_test))
    score, acc = model.evaluate(x_test, y_test,
                                batch_size=batch_size)
    print('Test score:', score)
    print('Test accuracy:', acc)
        
    if not os.path.exists(filepath):
        os.makedirs(filepath)
    model.save(os.path.join(filepath, 'model.h5'))

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [16]:
model.layers[0].output_shape

(None, None, 128)

In [11]:
x_test[:1].shape

(1, 100)

In [12]:
n_steps = 50
method = "gausslegendre"
return_convergence_delta = True
return_predictions = False
ig  = IntegratedGradients(model, layer=model.layers[0],
                          n_steps=n_steps, 
                          method=method,
                          return_convergence_delta=return_convergence_delta, 
                          return_predictions=return_predictions)

In [13]:
nb_samples = 5
bs = 10
x_test_red = x_test[:nb_samples]
test_labels_red = y_test[:nb_samples]

In [14]:
explanation = ig.explain(x_test_red, 
                         baselines=None, 
                         target=test_labels_red, 
                         internal_batch_size=bs)

(10, 100, 128)
<class 'tensorflow.python.framework.ops.EagerTensor'>
(10, 100, 128)
<class 'tensorflow.python.framework.ops.EagerTensor'>
(10, 100, 128)
<class 'tensorflow.python.framework.ops.EagerTensor'>
(10, 100, 128)
<class 'tensorflow.python.framework.ops.EagerTensor'>
(10, 100, 128)
<class 'tensorflow.python.framework.ops.EagerTensor'>
(10, 100, 128)
<class 'tensorflow.python.framework.ops.EagerTensor'>
(10, 100, 128)
<class 'tensorflow.python.framework.ops.EagerTensor'>
(10, 100, 128)
<class 'tensorflow.python.framework.ops.EagerTensor'>
(10, 100, 128)
<class 'tensorflow.python.framework.ops.EagerTensor'>
(10, 100, 128)
<class 'tensorflow.python.framework.ops.EagerTensor'>
(10, 100, 128)
<class 'tensorflow.python.framework.ops.EagerTensor'>
(10, 100, 128)
<class 'tensorflow.python.framework.ops.EagerTensor'>
(10, 100, 128)
<class 'tensorflow.python.framework.ops.EagerTensor'>
(10, 100, 128)
<class 'tensorflow.python.framework.ops.EagerTensor'>
(10, 100, 128)
<class 'tensorflow.

TypeError: 'NoneType' object cannot be interpreted as an integer